# 세상에 없는 얼굴 GAN, 오토인코더

In [8]:
# 배치 정규화, 이미지 확대, DCGAN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Reshape, Input, Flatten, Dropout
from tensorflow.keras.layers import Activation, LeakyReLU, UpSampling2D, Conv2D
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from tensorflow.keras.datasets import mnist

In [12]:
# 생성자 모델생성
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))  # 입력층 -> layer 1
generator.add(BatchNormalization())         # 배치정규화 : 평균 0, 분산이 1인 데이터로 재배치
generator.add(Reshape((7, 7, 128)))         # 3차원으로 데이터 타입 변경
generator.add(UpSampling2D())               # 이미지를 2배로 확장 시킴
generator.add(Conv2D(64, kernel_size=5, padding='same'))   # padding='same', 이미지 사이즈 줄지 않음
generator.add(BatchNormalization())         # 배치정규화 : 평균 0, 분산이 1인 데이터로 재배치
generator.add(Activation(LeakyReLU(0.2)))   # 활성화 함수 추가
generator.add(UpSampling2D())               # 이미지를 2배로 확장 시킴
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [ ]:
# 판별자 모델생성
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28, 28, 1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False